In [0]:
import json
from pathlib import Path
from functions.utility_functions import create_table_if_not_exists
import layer_01_bronze as bronze
import layer_02_silver as silver
import layer_03_gold as gold

# Variables
color                       = dbutils.widgets.get("color")
job_settings                = json.loads(dbutils.widgets.get("job_settings"))
table                       = job_settings['table']
settings                    = json.loads(next(Path().glob(f"./layer_*_{color}/{table}.json")).read_text())
dst_table_name              = settings.get("dst_table_name")

# Print job and table settings
settings_message = f"\n\nDictionary from {color}_settings.json:\n\n"
settings_message += json.dumps(job_settings, indent=4)
settings_message += f"\n\nContents of {table}.json:\n\n"
settings_message += json.dumps(settings, indent=4)
print(settings_message)

modules = {
    "bronze": bronze,
    "silver": silver,
    "gold": gold
}

# One function for pipeline
if "pipeline_function" in settings:
    modname, funcname = settings["pipeline_function"].split(".")
    pipeline_function = getattr(modules[modname], funcname)
    pipeline_function(spark, settings)

# Individual functions for each step
elif all(k in settings for k in ["reader_function", "transformer_function", "writer_function"]):
    modname, funcname = settings["reader_function"].split(".")
    reader_function = getattr(modules[modname], funcname)

    modname, funcname = settings["transformer_function"].split(".")
    transformer_function = getattr(modules[modname], funcname)

    modname, funcname = settings["writer_function"].split(".")
    writer_function = getattr(modules[modname], funcname)

    df = reader_function(spark, settings)
    df = transformer_function(spark, settings, df)
    create_table_if_not_exists(spark, df, dst_table_name)
    query = writer_function(spark, settings, df)
else:
    raise Exception(f"Could not find any ingest function name in settings.")